# data cleaning

turn tweet data set into workable dataframe using pyspark

In [164]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [165]:
tweets = sc.textFile("./data/10tweets.txt")

In [166]:
tweets_clean = tweets.flatMap(lambda x: x.split(' ')).filter(lambda x:x!='O').filter(lambda x:x!='').filter(lambda x:x!='<STOP>')


this operation is super slow

reducing the entire dataset to one string and then splitting it and re parallelizing 

can this be done while remaining an rdd ? 

In [168]:
tweets_string = tweets_clean.reduce(lambda x,y:x+' '+y)
import re
res = re.compile('\<START\>').split(tweets_string)
res = list(filter(None, res))
tweet_rdd = sc.parallelize(res)

group text to 1 tweet in rdd

find way to add key to each element

counter doesnt work in rdd 

In [169]:
tweets = tweet_rdd.zipWithIndex().map(lambda x: (x[1],x[0]))

### create data table compatible structure

- id , text , emojis

- tab seperate multiple emojis in one tweet

In [188]:
# get emojis out of text
emoji_rdd = tweets.map(lambda x: (x[0],re.findall(":.*?:",x[1])))
emoji_rdd_tab_sep = emoji_rdd.map(lambda x:(x[0],'\t'.join(x[1])))

text_rdd = tweets.map(lambda x: (x[0],re.sub(":.*?:","",x[1])))

df_rdd1 = text_rdd.leftOuterJoin(emoji_rdd_tab_sep).map(lambda x:(x[0],x[1][0],x[1][1]))

### duplicate tweets with multiple emojis into single emoji tweets

In [186]:
emoji_rdd1 = emoji_rdd.map(lambda x:(x[0],x[1][0]))
max_emoji = emoji_rdd.map(lambda x: len(x[1])).max()

for i in range(1,max_emoji):
    emoji_rdd2 = emoji_rdd.filter(lambda x:len(x[1])>i).map(lambda x: (x[0],x[1][i]))
    if i==1:
        emoji_rdd3 = emoji_rdd1.union(emoji_rdd2)
    else:
        emoji_rdd3 = emoji_rdd3.union(emoji_rdd2)

df_rdd2 = text_rdd.leftOuterJoin(emoji_rdd3).map(lambda x:(x[0],x[1][0],x[1][1]))

In [172]:
from pyspark.sql import SQLContext

s   = SQLContext(sc)
df = s.createDataFrame(df_rdd2, ['id', 'text','emoji'])

df.repartition(1).write.csv('./data/outv4', sep=',', header=True)
